In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from openpyxl import load_workbook
import os
from openpyxl.styles import PatternFill
from moviepy.editor import VideoFileClip

In [3]:
def remove_audio(video_path, output_path):
    video_clip = VideoFileClip(video_path)
    
    video_clip = video_clip.without_audio()

    video_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

    video_clip.close()


def check_video_properties(video_path, video_path2):
    original_clip = VideoFileClip(video_path)

    original_duration = original_clip.duration
    original_fps = original_clip.fps
    original_resolution = original_clip.size

    original_clip.close()

    modified_clip = VideoFileClip(video_path2)

    modified_duration = modified_clip.duration
    modified_fps = modified_clip.fps
    modified_resolution = modified_clip.size

    modified_clip.close()

    if original_duration == modified_duration:
        print("Duration preserved.")
        print("duration == ", modified_duration)
    else:
        print("Duration not preserved.")

    if original_fps == modified_fps:
        print("Frame rate (fps) preserved.")
        print("fps == ", modified_fps)
    else:
        print("Frame rate (fps) not preserved.")

    if original_resolution == modified_resolution:
        print("Resolution preserved.")
        print("resolution == ", modified_resolution)
    else:
        print("Resolution not preserved.")


In [42]:
inp = input("Enter file name: ", )

input_video_path = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Video Data\More Video Data\Subjects\S6(Ram)\%s.MP4" % (inp)
output_video_path = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Processed Videos\More_Processed_Videos\S6(Ram)\%s_no_audio.mp4" % (inp)
excel_path = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Processed Videos\coordinate_excels\More_coordinate_excels\S6(Ram)\%s_no_audio.xlsx" % (inp)

remove_audio(input_video_path, output_video_path)
check_video_properties(input_video_path, output_video_path)

Enter file name: GX010358
Moviepy - Building video C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Processed Videos\More_Processed_Videos\S6(Ram)\GX010358_no_audio.mp4.
Moviepy - Writing video C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Processed Videos\More_Processed_Videos\S6(Ram)\GX010358_no_audio.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\CS - 299 (Sem2) Project\Processed Videos\More_Processed_Videos\S6(Ram)\GX010358_no_audio.mp4
Duration preserved.
duration ==  31.59
Frame rate (fps) preserved.
fps ==  200.0
Resolution preserved.
resolution ==  [1920, 1080]


In [43]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

all_coordinates = []

cap = cv2.VideoCapture(output_video_path)
count = 0
heel_strike_1_frame = None
heel_strike_2_frame = None
frame_rate = 200  # FPS
heel_strike_threshold = 0.1  # Adjust as needed
width = 1280
height = 720

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if frame is None:
            break
        coordinates = []
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image, (width, height))
        image.flags.writeable = False

        results = pose.process(image)
        count = count + 1

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x))
        coordinates.append((landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y))

        # Extracting coordinates of specific landmarks
        left_heel_y = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y if landmarks[
            mp_pose.PoseLandmark.LEFT_HEEL.value].visibility > 0 else None
        right_heel_y = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y if landmarks[
            mp_pose.PoseLandmark.RIGHT_HEEL.value].visibility > 0 else None

        # Detecting heel strikes
        if left_heel_y is not None and right_heel_y is not None:
            if abs(left_heel_y - right_heel_y) > heel_strike_threshold:
                # Assuming the highest point among left and right heel y-coordinates is the heel strike
                heel_strike_frame = count
                if left_heel_y > right_heel_y:
                    heel_strike_side = "Left"
                else:
                    heel_strike_side = "Right"
                
                if heel_strike_1_frame is None:
                    heel_strike_1_frame = heel_strike_frame
                    heel_strike_1_side = heel_strike_side
                else:
                    heel_strike_2_frame = heel_strike_frame
                    heel_strike_2_side = heel_strike_side

        # Appending coordinates to the list
        all_coordinates.append(coordinates)

        # Drawing landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        # Highlighting heel strike frames in red
        if count == heel_strike_1_frame or count == heel_strike_2_frame:
            cv2.rectangle(image, (0, 0), (50, 50), (0, 0, 255), -1)  # Red rectangle in the corner

        # Set up the window
        cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)

        # Resize the window to the desired dimensions
        cv2.resizeWindow('Mediapipe Feed', width, height)  # Specify the desired width and height

        # Display the image in the resized window
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Calculating duration of heel strikes
heel_strike_1_duration = heel_strike_1_frame / frame_rate if heel_strike_1_frame is not None else None
heel_strike_2_duration = heel_strike_2_frame / frame_rate if heel_strike_2_frame is not None else None

print("First Heel Strike Frame:", heel_strike_1_frame)
print("First Heel Strike Side:", heel_strike_1_side)
print("Second Heel Strike Frame:", heel_strike_2_frame)
print("Second Heel Strike Side:", heel_strike_2_side)
print("Duration of First Heel Strike:", heel_strike_1_duration, "seconds")
print("Duration of Second Heel Strike:", heel_strike_2_duration, "seconds")

# Saving coordinates to Excel with heel strike frames marked
df = pd.DataFrame(all_coordinates)
df.to_excel(excel_path, index=False)

if os.path.exists(excel_path):
    os.system("TASKKILL /F /IM EXCEL.EXE") 
# Open the workbook
wb = load_workbook(excel_path)
ws = wb.active

# Define a red fill
red_fill = PatternFill(start_color='FFFF0000',
                       end_color='FFFF0000',
                       fill_type='solid')

# Mark heel strike frames in Excel
if heel_strike_1_frame:
    ws.cell(row=heel_strike_1_frame + 300, column=1).fill = red_fill
if heel_strike_2_frame:
    ws.cell(row=heel_strike_2_frame - 300, column=1).fill = red_fill

# Save the modified workbook
wb.save(excel_path)

cap.release()
cv2.destroyAllWindows()


First Heel Strike Frame: 511
First Heel Strike Side: Left
Second Heel Strike Frame: 5314
Second Heel Strike Side: Left
Duration of First Heel Strike: 2.555 seconds
Duration of Second Heel Strike: 26.57 seconds
